In [2]:
from langchain_openai import OpenAI,ChatOpenAI

model=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.3
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import Iterator,List

prompt=ChatPromptTemplate.from_template(
    "响应以CSV的格式返回中文列表，不要返回其他内容。请输出与{transportation}类似的交通工具。"
)

In [4]:
str_chain=prompt | model | StrOutputParser()

In [5]:
str_chain.invoke({"transportation":"飞机"})

'```csv\n飞机,火车,轮船,汽车\n```'

In [6]:
for chunk in str_chain.stream({"transportation":"飞机"}):
    print(chunk,end="",flush=True)

```csv
飞机,火车,轮船,汽车
```

In [7]:
def split_into_list(input:Iterator[str])->Iterator[List[str]]:
    buffer=""     # buffer 用于存储还没凑够一个逗号的残留文本
    for chunk in input:
        buffer+=chunk   # 1. 收到模型传来的新碎片，加到缓冲区
        while "," in buffer:    
            comma_index=buffer.index(",")   #找到第一个逗号的位置
            # 3. 产生（yield）逗号前面的内容，并去掉空格
            # 这样下游能立刻收到一个列表元素
            yield [buffer[:comma_index].strip()]
            buffer=buffer[comma_index+1:]   # 4. 把已经处理过的内容（连同逗号）从缓冲区删掉
    yield [buffer.strip()]    # 5. 当模型说完话后，把缓冲区剩下的最后一部分产出

In [8]:
list_chain=str_chain | split_into_list
for chunk in list_chain.stream({"transportation":"飞机"}):
    print(chunk,end="",flush=True)

['```csv\n飞机']['火车']['轮船']['汽车']['自行车']['公共汽车']['巴士']['卡车']['摩托车']['拖拉机\n```']

异步版本

In [10]:
from typing import AsyncIterator

async def asplit_into_list(input:AsyncIterator[str])->AsyncIterator[List[str]]:
    buffer=""     # buffer 用于存储还没凑够一个逗号的残留文本
    async for chunk in input:
        buffer+=chunk   # 1. 收到模型传来的新碎片，加到缓冲区
        while "," in buffer:    
            comma_index=buffer.index(",")   #找到第一个逗号的位置
            # 3. 产生（yield）逗号前面的内容，并去掉空格
            # 这样下游能立刻收到一个列表元素
            yield [buffer[:comma_index].strip()]
            buffer=buffer[comma_index+1:]   # 4. 把已经处理过的内容（连同逗号）从缓冲区删掉
    yield [buffer.strip()]    # 5. 当模型说完话后，把缓冲区剩下的最后一部分产出
    
list_chain=str_chain | asplit_into_list

In [11]:
async for chunk in list_chain.astream({"transportation":"飞机"}):
    print(chunk,end="",flush=True)

['```csv\n飞机']['火车']['轮船']['汽车']['自行车\n```']

In [14]:
await list_chain.ainvoke({"transportation":"飞机"})

['```csv\n飞机\n高铁\n轮船\n汽车\n火车\n```']